In [4]:
import pandas as pd
import numpy as np

from util import comparar_columnas, eliminar_caracteristicas, prueba_umbral
from feature_selection import *

In [5]:
final = pd.read_csv('final_dataframe.csv', index_col=False)

In [6]:
corr_features = [
    'alpha.0-5cm', 'ksat_0-5cm', 'PIRange_Sand.0-5cm', 'alpha_0-5cm', 
    'ksat_5-15cm', 'PIRange_Sand.15-30cm', 'alpha_5-15cm', 'ksat_15-30cm', 
    'PIRange_Sand.60-100cm', 'alpha_15-30cm', 'ksat_60-100cm', 'PP', 
    'AvMoist.0-5cm', 'ksat_100-200cm', 'PWP.0-5cm', 'AvMoist.5-15cm', 
    'n_0-5cm', 'PWP.100-200cm', 'AvMoist.60-100cm', 'n_5-15cm', 'slope', 
    'Bulkd.0-5cm', 'n_15-30cm', 'Tex_Class.0-5cm', 'Bulkd.5-15cm', 
    'n_60-100cm', 'Tex_Class.5-15cm', 'Bulkd.60-100cm', 'n_100-200cm', 
    'Tex_Class.15-30cm', 'Clay.0-5cm', 'PIRange_Bulkd.0-5cm', 
    'Tex_Class.100-200cm', 'Clay.5-15cm', 'PIRange_Bulkd.5-15cm', 
    'theta_s_0-5cm', 'Clay.15-30cm', 'PIRange_Bulkd.60-100cm', 
    'theta_s_5-15cm', 'FC.0-5cm', 'PIRange_Clay.0-5cm', 'theta_s_15-30cm', 
    'FC.5-15cm', 'PIRange_Clay.5-15cm', 'theta_s_60-100cm', 'FC.60-100cm', 
    'PIRange_Clay.15-30cm', 'VMoist', 'Valor'
]
corr_features[47] = 'valor_humedad_suelo1'
final.columns = final.columns.str.replace('.tif', '')
features = ['PP',
'n_5-15cm',
'slope',
'Bulkd.5-15cm',
'n_100-200cm',
'Tex_Class.100-200cm',
'theta_s_0-5cm',
'theta_s_15-30cm',
'valor_humedad_suelo1',
'Valor']
final = final[features]

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import stats

# Grid de hiperparámetros extendido para RandomForest
param_grid = {
    # Parámetros relacionados con la estructura de los árboles
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    
    # Parámetros de aleatoriedad y features
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'random_state': [42],
    
    # Parámetros de regularización y peso
    'max_leaf_nodes': [None, 50, 100, 200],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
    'min_impurity_decrease': [0.0, 0.01, 0.05],
    
    # Parámetros de paralelización
    'n_jobs': [-1]  # Usa todos los cores disponibles
}

def train_rf_with_confidence(X_train, X_test, y_train, y_test, confidence_level=0.95):
    """
    Entrena un RandomForest usando GridSearchCV y calcula el intervalo de confianza
    del error (1 - accuracy)
    
    Parameters:
    -----------
    X_train : array-like
        Datos de entrenamiento
    X_test : array-like
        Datos de prueba
    y_train : array-like
        Etiquetas de entrenamiento
    y_test : array-like
        Etiquetas de prueba
    confidence_level : float, optional (default=0.95)
        Nivel de confianza para el intervalo
        
    Returns:
    --------
    dict : Diccionario con los mejores parámetros, error y su intervalo de confianza
    """
    # Crear el modelo base
    rf = RandomForestClassifier()
    
    # Configurar la búsqueda de grid con validación cruzada
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=3,
        scoring='accuracy',
        verbose=2,
        n_jobs=-1
    )
    
    # Ajustar el modelo
    grid_search.fit(X_train, y_train)
    
    # Obtener predicciones en el conjunto de prueba
    y_pred = grid_search.predict(X_test)
    
    # Calcular el error (1 - accuracy) para cada predicción
    errors = y_pred != y_test
    error_rate = np.mean(errors)
    
    # Calcular el intervalo de confianza
    n = len(y_test)
    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * np.sqrt((error_rate * (1 - error_rate)) / n)
    
    results = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'test_error': error_rate,
        'confidence_interval': margin_of_error,
        'error_range': f"{error_rate:.4f} ± {margin_of_error:.4f}"
    }
    
    return results

target_column = 'Valor'
X = final.drop(target_column, axis=1)
y = final[target_column]
    
    # Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
results = train_rf_with_confidence(X_train, X_test, y_train, y_test)
print("Mejores parámetros:", results['best_params'])
print(f"Error en test con intervalo de confianza del 95%: {results['error_range']}")


Fitting 3 folds for each of 151200 candidates, totalling 453600 fits
Mejores parámetros: {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': -1, 'random_state': 42}
Error en test con intervalo de confianza del 95%: 0.1095 ± 0.0244


In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np
from scipy import stats

# Grid de hiperparámetros extendido para XGBoost
param_grid = {
    # Parámetros de estructura del árbol
    'max_depth': [3, 5, 7, 9, 11],
    #'min_child_weight': [1, 3, 5, 7],
    #'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    
    # Parámetros de regulación
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'n_estimators': [100, 200, 300, 500],
    #'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    #'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    
    # Parámetros de regularización
    #'reg_alpha': [0, 0.1, 0.5, 1.0],
    #'reg_lambda': [0.1, 0.5, 1.0, 2.0],
    
    
    # Otros parámetros
    'random_state': [42]
}

def train_xgb_with_confidence(X_train, X_test, y_train, y_test, confidence_level=0.95):
    """
    Entrena un XGBoost usando GridSearchCV y calcula el intervalo de confianza
    del error (1 - accuracy)
    
    Parameters:
    -----------
    X_train : array-like
        Datos de entrenamiento
    X_test : array-like
        Datos de prueba
    y_train : array-like
        Etiquetas de entrenamiento
    y_test : array-like
        Etiquetas de prueba
    confidence_level : float, optional (default=0.95)
        Nivel de confianza para el intervalo
        
    Returns:
    --------
    dict : Diccionario con los mejores parámetros, error y su intervalo de confianza
    """
    # Crear el modelo base
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, verbosity=2)
    
    # Configurar la búsqueda de grid con validación cruzada
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        cv=3,
        scoring='accuracy',
        verbose=3,
        n_jobs=-1
    )
    
    # Ajustar el modelo
    grid_search.fit(X_train, y_train)
    
    # Obtener predicciones en el conjunto de prueba
    y_pred = grid_search.predict(X_test)
    
    # Calcular el error (1 - accuracy) para cada predicción
    errors = y_pred != y_test
    error_rate = np.mean(errors)
    
    # Calcular el intervalo de confianza
    n = len(y_test)
    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * np.sqrt((error_rate * (1 - error_rate)) / n)
    
    results = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'test_error': error_rate,
        'confidence_interval': margin_of_error,
        'error_range': f"{error_rate:.4f} ± {margin_of_error:.4f}"
    }
    
    return results

target_column = 'Valor'
X = final.drop(target_column, axis=1)
y = final[target_column]
    
    # Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
results = train_xgb_with_confidence(X_train, X_test, y_train, y_test)
print("Mejores parámetros:", results['best_params'])
print(f"Error en test con intervalo de confianza del 95%: {results['error_range']}")

Fitting 3 folds for each of 120 candidates, totalling 360 fits


c:\Users\ivo\.conda\envs\geotiff\lib\site-packages\xgboost\core.py:158: UserWarning: [18:36:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parámetros: {'learning_rate': 0.05, 'max_depth': 9, 'n_estimators': 500, 'random_state': 42}
Error en test con intervalo de confianza del 95%: 0.1032 ± 0.0238


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np
from scipy import stats

# Grid de hiperparámetros extendido para SVC
param_grid_svc = {
    'C': [0.1, 1, 10, 100],  # Parámetro de penalización
    'kernel': ['rbf'],  # Tipos de kernel
    'gamma': ['scale', 'auto', 0.01, 0.1, 1,10,20,30,50,100],  # Parámetro del kernel RBF
    'random_state': [42],  # Reproducibilidad
}

def train_svc_with_confidence(X_train, X_test, y_train, y_test, confidence_level=0.95):
    """
    Entrena un modelo SVC usando GridSearchCV y calcula el intervalo de confianza
    del error (1 - accuracy)
    
    Parameters:
    ----------- 
    X_train : array-like
        Datos de entrenamiento
    X_test : array-like
        Datos de prueba
    y_train : array-like
        Etiquetas de entrenamiento
    y_test : array-like
        Etiquetas de prueba
    confidence_level : float, optional (default=0.95)
        Nivel de confianza para el intervalo
        
    Returns:
    --------
    dict : Diccionario con los mejores parámetros, error y su intervalo de confianza
    """
    # Crear el modelo base
    svc = SVC()
    
    # Configurar la búsqueda de grid con validación cruzada
    grid_search = GridSearchCV(
        estimator=svc,
        param_grid=param_grid_svc,
        cv=3,
        scoring='accuracy',
        verbose=2,
        n_jobs=-1
    )
    
    # Ajustar el modelo
    grid_search.fit(X_train, y_train)
    
    # Obtener predicciones en el conjunto de prueba
    y_pred = grid_search.predict(X_test)
    
    # Calcular el error (1 - accuracy) para cada predicción
    errors = y_pred != y_test
    error_rate = np.mean(errors)
    
    # Calcular el intervalo de confianza
    n = len(y_test)
    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * np.sqrt((error_rate * (1 - error_rate)) / n)
    
    results = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'test_error': error_rate,
        'confidence_interval': margin_of_error,
        'error_range': f"{error_rate:.4f} ± {margin_of_error:.4f}"
    }
    
    return results

# Ejemplo de uso
target_column = 'Valor'
X = final.drop(target_column, axis=1)
y = final[target_column]

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Entrenar el modelo
results = train_svc_with_confidence(X_train, X_test, y_train, y_test)
print("Mejores parámetros:", results['best_params'])
print(f"Error en test con intervalo de confianza del 95%: {results['error_range']}")
